In [ ]:
stripepath={
        'Zebra': "./stripe_results/GM12878.bedpe",
        'StripeNN': './stripe_results/results_filtered.tsv',
        'Quagga': './stripe_results/5000nt_MR10000000_ML50000_MD7000000_MW50000_WS7_SG1RH0.3_NSTR10_ALL_hg38GM12878_completedfiltered.txt'
}

In [ ]:
# %%bash

# awk '{
#     if ($4 == 2 || $4 == 6 || $4 == 7) {
#         $4 = 1;
#         print;
#     }
# }' /nfs/turbo/umms-drjieliu/proj/ENCODE/ChromHMM/GM12878_GMM_liftover_hg38/E116_15_coreMarks_hg38lift_stateno.bed > ./HMM_anno_peaks/GM12878_HMM_hg38_EPanno_.bed 

In [ ]:
def _zebra_read_stripes(file_name):
    
    hold_dict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}
    source_dict = {5:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]},3:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,_,c4,c5=line.split()
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
            if int(c2)-int(c1)<int(c5)-int(c4):
                source_dict[5][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]))]
            else:
                source_dict[3][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]))]
                
    return source_dict, hold_dict


def _stripeNN_read_stripes(file_name):
    
    hold_dict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}
    source_dict = {5:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]},3:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,_,c4,c5,l,w,mean,maxpx,pval,stripiness=line.rstrip("\n").split("\t")
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
            if int(c2)-int(c1)<int(c5)-int(c4):
                source_dict[5][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[l,w,mean,maxpx,pval,stripiness]))]
            else:
                source_dict[3][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[l,w,mean,maxpx,pval,stripiness]))]
                
    return source_dict, hold_dict

def _read_stripes(file_name):
    
    hold_dict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}
    source_dict = {5:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]},3:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,_,c4,c5,pval=line.rstrip("\n").split()
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
            if int(c2)-int(c1)<int(c5)-int(c4):
                source_dict[5][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[float(pval)]))]
            else:
                source_dict[3][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[float(pval)]))]
                
    return source_dict, hold_dict

def _read_peaks(file_name):
    
    hold_pdict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]+["chrY"]}
    source_pdict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]+["chrY"]}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"] + ["chrY"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,payload=line.split()
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
#             if int(c2)-int(c1)<int(c5)-int(c4):
            source_pdict[chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,payload]]))]
                
    return source_pdict, hold_pdict



def IntersectingIntervals(arr1, w_1, arr2, spacer=50000):
    i = j = 0
    n = len(arr1)
    m = len(arr2)
        
    result=[]
    while i < n and j < m:
        while (j < m and arr2[j][3]<arr1[i][w_1]):
            j+=1
            
        if j>=m: #one level up?
            break
        
        if arr1[i][w_1]-spacer<=arr2[j][2]<=arr1[i][w_1+1]+spacer and arr1[i][w_1]-spacer<=arr2[j][3]<=arr1[i][w_1+1]+spacer and arr2[j][-1]!=0:
            result+=[arr1[i]]
            i+=1
        elif arr2[j][2]>arr1[i][w_1+1]+spacer:
            i+=1
        else:
            j+=1
        
    return result


def _load_Intersect(source_dict,source_peaks,hold_dict,spacer):
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        hold_dict[key]=IntersectingIntervals(source_dict[5][key],2,source_peaks[key],spacer=2000)
        hold_dict[key]+=IntersectingIntervals(source_dict[3][key],4,source_peaks[key],spacer=2000)
        
def _write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            payload = value[-1]
            c5=value[-2]
            c4=value[-3]
            write_out+="\t".join([str(y) for y in value[1:4]]+[value[1]]+[str(c4)]+[str(c5)]+[str(payload)])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)
        
        
def _z_write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]

            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)
        
def _snn_write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]
            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)
        
def call_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=50000):
    sdict, hdict=_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _write_Intersect(hdict, fileoutname)
    
def z_call_CTCF_stripes(stripename, ctcfname, fileoutname):
    sdict, hdict=_zebra_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _z_write_Intersect(hdict, fileoutname)
    
def snn_call_CTCF_stripes(stripename, ctcfname, fileoutname):
    sdict, hdict=_stripeNN_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _snn_write_Intersect(hdict, fileoutname)

In [ ]:

def noIntersectingIntervals(arr1, w_1, arr2, spacer=50000):
    i = j = 0
    n = len(arr1)
    m = len(arr2)
        
    result=[]
    while i < n and j < m:
        while (j < m and arr2[j][3]<arr1[i][w_1]):
            j+=1
            
        if j>=m: #one level up?
            break
        
        if arr1[i][w_1]-spacer<=arr2[j][2]<=arr1[i][w_1+1]+spacer and arr1[i][w_1]-spacer<=arr2[j][3]<=arr1[i][w_1+1]+spacer and arr2[j][-1]!=0:
            i+=1
        elif arr2[j][2]>arr1[i][w_1+1]+spacer:
            result+=[arr1[i]]
            i+=1
        else:
            j+=1
        
    return result

def _load_noIntersect(source_dict,source_peaks,hold_dict,spacer=2000):
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        hold_dict[key]=noIntersectingIntervals(source_dict[5][key],2,source_peaks[key],spacer)
        hold_dict[key]+=noIntersectingIntervals(source_dict[3][key],4,source_peaks[key],spacer)
        
def _write_noIntersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            payload = value[-1]
            c5=value[-2]
            c4=value[-3]
            write_out+="\t".join([str(y) for y in value[1:4]]+[value[1]]+[str(c4)]+[str(c5)]+[str(payload)])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)

def _z_write_noIntersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]

            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)

def _snn_write_noIntersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]
            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)


def call_non_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=1000):
    sdict, hdict=_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_noIntersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _write_noIntersect(hdict, fileoutname)

def z_call_non_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=1000):
    sdict, hdict=_zebra_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_noIntersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _z_write_noIntersect(hdict, fileoutname)
    
def snn_call_non_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=1000):
    sdict, hdict=_stripeNN_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_noIntersect(sdict, source_pdict,hdict,spacer)
        #noIntersectingIntervals(sdict[5]['chr1'],2,source_pdict['chr1'],spacer=5000))
    _snn_write_noIntersect(hdict, fileoutname)

In [ ]:
H1_CTCF_PATH="./factors/JASPAR/sorted_H1_CTCF.bed"
GM12878_CTCF_PATH="./factors/JASPAR/sorted_GM12878_CTCF.bed"

In [ ]:
H1_EP_PATH="./factors/H1_HMM_hg38_EPanno.bed"
GM12878_EP_PATH="./factors/GM12878_HMM_hg38_EPanno.bed"

In [ ]:
# CTCF containing stripes from source

call_CTCF_stripes("./strip_results/5000nt_MR10000000_ML50000_MD7000000_MW50000_WS7_SG1RH0.3_NSTR10_ALL_hg38GM12878_completedfiltered.txt",\
GM12878_CTCF_PATH,"Quagga_GM12878_CTCFonly.txt")

z_call_CTCF_stripes("./strip_results/sorted_Zebra_GM12878.bedpe",\
                        GM12878_CTCF_PATH,"Zebra_GM12878_CTCFonly.txt")

snn_call_CTCF_stripes('./strip_results/sorted_stripeNN_results_filtered.tsv',\
                        GM12878_CTCF_PATH,"StripeNN_GM12878_CTCFonly.txt")

In [ ]:
# return stripes excluded from CTCF from source

call_non_CTCF_stripes("./strip_results/5000nt_MR10000000_ML50000_MD7000000_MW50000_WS7_SG1RH0.3_NSTR10_ALL_hg38GM12878_completedfiltered.txt",\
                 GM12878_CTCF_PATH,"./Quagga_GM12878_noCTCF.txt",spacer=50000)

z_call_non_CTCF_stripes("./strip_results/sorted_Zebra_GM12878.bedpe",\
                 GM12878_CTCF_PATH,"Zebra_GM12878_noCTCF.txt",spacer=50000)

snn_call_non_CTCF_stripes("./strip_results/StripeNN_GM12878_noCTCF.txt",\
                 GM12878_CTCF_PATH,"StripeNN_GM12878_noCTCF.txt",spacer=50000)

In [ ]:
# Intersect "noCTCF" with EP regions

call_CTCF_stripes("Quagga_G12878_noCTCF.txt",\
                        GM12878_EP_PATH,"./All_Intersect_EP/Quagga_GM12878_Allstripes_EP.txt",spacer=5000)

z_call_CTCF_stripes("Zebra_GM12878_noCTCF.txt",\
                        GM12878_EP_PATH,"Zebra_GM12878_EP.txt")

snn_call_CTCF_stripes("StripeNN_GM12878_noCTCF.txt",\
                       GM12878_EP_PATH,"StripeNN_GM12878_EP.txt")

In [ ]:
#exclude noCTCF stripes from EP regions for unassigned

call_non_CTCF_stripes("./Quagga_GM12878_noCTCF.txt",\
                 GM12878_EP_PATH,"./Quagga_GM12878_Unassigned.txt",spacer=2000)

z_call_non_CTCF_stripes("./Zebra_AB_Merged_GM12878_noCTCF.txt",\
                 GM12878_EP_PATH,"./Zebra_AB_GM12878_Unassigned.txt",spacer=2000)

snn_call_non_CTCF_stripes("./StripeNN_GM12878_noCTCF.txt",\
                 GM12878_EP_PATH,"./StripeNN_GM12878_Unassigned.txt",spacer=2000)

In [ ]:
# Do it again, for H1 Hi-C versus Micro-C

In [ ]:
#H1 Micro-C
call_CTCF_stripes("/stripe_results/H1_microC_vs_HiC/5000nt_MR5000000_ML20000_MD500000_WS20_MW_50000_SG6RH0.1_NSTR10_alt_ALL_H1_microC_hg38_filtered.txt",\
                 H1_CTCF_PATH,"H1-hESC_MiC_CTCFonly.txt",spacer=5000)

call_non_CTCF_stripes("/stripe_results/H1_microC_vs_HiC/5000nt_MR5000000_ML20000_MD500000_WS20_MW_50000_SG6RH0.1_NSTR10_alt_ALL_H1_microC_hg38_filtered.txt",\
                 H1_CTCF_PATH,"H1-hESC_MiC_noCTCF.txt",spacer=5000)

call_CTCF_stripes("H1-hESC_MiC_noCTCF.txt",\
                 H1_EP_PATH,"H1-hESC_MiC_EP.txt",2000)

call_non_CTCF_stripes("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/ctcf_deficient/Othermodels/Quagga_H1_MicroC_noCTCF.txt",\
                 H1_EP_PATH,"Quagga_H1_MiC_Unassigned.txt",spacer=2000)


#H1 Hi-C
call_CTCF_stripes("/stripe_results/H1_microC_vs_HiC/5000nt_MR15000000_ML20000_MD5000000_MW50000_WS10_SG1RH0.1_NSTR0_ALL_CHR.hg38H1.filtered.txt",\
                 H1_CTCF_PATH,"H1-hESC_HiC_CTCFonly.txt",spacer=5000)

call_non_CTCF_stripes("./stripe_results/H1_microC_vs_HiC/5000nt_MR15000000_ML20000_MD5000000_MW50000_WS10_SG1RH0.1_NSTR0_ALL_CHR.hg38H1.filtered.txt",\
                 H1_CTCF_PATH,"H1-hESC_HiC_noCTCF.txt",spacer=5000)

call_CTCF_stripes("H1-hESC_MiC_noCTCF.txt",\
                 H1_EP_PATH,"H1-hESC_HiC_EP.txt",2000)

call_non_CTCF_stripes("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/ctcf_deficient/Othermodels/Quagga_H1_MicroC_noCTCF.txt",\
                 H1_EP_PATH,"Quagga_H1_HiC_Unassigned.txt",spacer=2000)
